# 2MRS Group Catalogs

In this notebook we will explore the various 2MRS group catalogs and look at the most massive halos. We'll then compile the group catalogs to be used in the data analysis.

In [40]:
import os, sys
import math
#sys.path.append("/zfs/nrodd/NPTF-ID-Catalog/2MASS/")
# sys.path.append("/group/hepheno/smsharma/Fermi-LSS/2MASS/")


import numpy as np
import pandas as pd
import healpy as hp
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13, z_at_value
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from astropy.io import fits
from tqdm import *
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.cosmology import Planck15, z_at_value
from scipy.optimize import fsolve

from units import *
# import tools
# from tools import lb2pix
import halo as hl
import Jpdf
import JBpdf
import Dpdf

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Huchra et al catalog (original 2MRS)

Huchra (original 2MASS) catalog. We don't actually do anything with it, just look at the 2MASS IDs etc to confirm the format.

In [24]:
# fits.open("/group/hepheno/smsharma/Fermi-LSS/AdditionalData/catalog/2mrs_1175_done.fits")[1].data['TMID']

## Cosmicflow

Cosmicflow catalog. Has a bunch of info that we use like common cluster names.

In [25]:
from astropy.io import ascii
cf_df = ascii.read("../DataFiles/Misc/ajaa259at2_mrt.txt").to_pandas()
cf_df.columns.values

array(['Nest', 'o_<DM>-gp', '<DM>-gp', 'e_<DM>-gp', '<Dist>-gp', 'Abell',
       'GName', 'Npv', 'PGC1', 'GLON-gp', 'GLAT-gp', 'SGLON-gp',
       'SGLAT-gp', 'logLKs', 'cf', 'Vsigma-proj', 'R2t-proj',
       '<Vhelio>-gp', '<Vgsr>-gp', '<Vls>-gp', '<Vcmb>-g', '<Vcmba>-gp',
       'Vrms-gp', 'Mvir', 'Mlum', 'LDC', 'HDC', '2M++', 'MK2011', 'Icnt'], dtype=object)

In [26]:
cf_df[cf_df['Nest'] == 100350][['<Vhelio>-gp', '<Vgsr>-gp', '<Vls>-gp', '<Vcmb>-g', '<Vcmba>-gp']]

,<Vhelio>-gp,<Vgsr>-gp,<Vls>-gp,<Vcmb>-g,<Vcmba>-gp
43,532.0,351.0,275.0,783.0,785.0


## Tully 3500

In [27]:
tully_3500 = ascii.read("../DataFiles/Misc/apjaa76dbt2_mrt.txt").to_pandas()

In [28]:
tully_3500.head()

,PGC1,PGC1+,Mem,GLON,GLAT,SGL,SGB,Ksmag,logK,Vh,VLS,ND,D,e_D,sigmaL,sigmaV,R2t,Rg,logMK,logMd
0,43296.0,43296.0,191.0,302.2241,21.6465,156.3251,-11.5819,4.61,12.71,3407.0,3142.0,59.0,36.96,3.0,595.0,800.0,1.612,0.893,14.624,14.717
1,46618.0,43296.0,30.0,307.8738,19.2865,159.6426,-6.8008,6.07,12.11,3340.0,3086.0,5.0,37.57,7.0,350.0,307.0,0.950,0.745,13.936,13.808
2,45174.0,43296.0,34.0,306.0424,32.5707,146.1941,-6.0422,6.42,11.96,3292.0,3059.0,4.0,44.03,8.0,307.0,300.0,0.833,0.693,13.764,13.756
3,40498.0,43296.0,24.0,297.5648,23.0823,153.9019,-15.4648,6.48,11.92,3258.0,2986.0,4.0,28.45,6.0,296.0,170.0,0.801,0.607,13.713,13.207
4,43557.0,43296.0,22.0,302.9333,36.4075,141.8602,-7.7062,6.82,11.81,3296.0,3066.0,7.0,39.89,8.0,267.0,163.0,0.724,0.428,13.582,13.017


In [29]:
tully_gal_3500 = ascii.read("../DataFiles/Misc/apjaa76dbt3_mrt.txt").to_pandas()

In [30]:
tully_gal_3500.head()

,PGC,Name,RAdeg,DEdeg,GLON,GLAT,SGL,SGB,T,Bmag,Ksmag,logK,Vh,VLS,D,e_D,PGC1
0,13418.0,NGC1399,54.6212,-35.4506,236.7163,-53.6356,262.5460,-42.0776,-4.6,10.35,6.44,11.25,1425.0,1283.0,22.08,11.0,13418.0
1,13179.0,NGC1365,53.4016,-36.1405,237.9564,-54.5979,261.8963,-40.9741,3.2,9.83,6.59,11.19,1664.0,1523.0,16.98,8.0,13418.0
2,13433.0,NGC1404,54.7163,-35.5942,236.9551,-53.5547,262.3357,-42.1253,-4.8,10.81,6.90,11.07,1873.0,1730.0,18.79,10.0,13418.0
3,13318.0,NGC1380,54.1149,-34.9761,235.9261,-54.0585,263.2770,-41.7608,-2.3,10.79,6.96,11.05,1855.0,1716.0,18.62,7.0,13418.0
4,13344.0,NGC1387,54.2377,-35.5066,236.8246,-53.9462,262.5532,-41.7605,-2.9,11.70,7.52,10.82,1265.0,1124.0,19.14,13.0,13418.0


In [31]:
tully_3500.sort_values(['D']).head()

,PGC1,PGC1+,Mem,GLON,GLAT,SGL,SGB,Ksmag,logK,Vh,VLS,ND,D,e_D,sigmaL,sigmaV,R2t,Rg,logMK,logMd
3073,5064336.0,2557.0,16.0,359.2117,-0.6057,186.4285,41.4388,1.89,10.56,9.0,-36.0,16.0,0.01,3.0,89.0,78.0,0.241,1.544,12.146,12.935
3081,6830.0,2557.0,1.0,272.1595,-68.9493,254.2878,-20.8612,11.30,6.79,-52.0,-159.0,1.0,0.42,6.0,16.0,0.0,0.043,NaN,9.901,NaN
3086,4713564.0,2557.0,1.0,214.8530,43.6613,78.4018,-38.7518,14.00,5.71,39.0,-82.0,1.0,0.42,5.0,10.0,0.0,0.028,NaN,9.362,NaN
3074,63616.0,2557.0,1.0,25.3411,-18.3951,229.0730,57.0920,7.26,8.41,-57.0,52.0,1.0,0.48,6.0,30.0,0.0,0.080,NaN,10.709,NaN
3075,3844.0,2557.0,1.0,129.7378,-60.5774,299.1522,-1.7785,7.50,8.31,-232.0,-97.0,1.0,0.74,5.0,28.0,0.0,0.077,NaN,10.661,NaN


In [32]:
tully_gal_3500.sort_values(['D']).head()

,PGC,Name,RAdeg,DEdeg,GLON,GLAT,SGL,SGB,T,Bmag,Ksmag,logK,Vh,VLS,D,e_D,PGC1
7660,5064336.0,Milky Way,266.4167,-29.0078,359.9443,-0.0461,185.8365,42.2483,NaN,NaN,-8.4,10.51,-8.0,-34.0,0.01,1.0,5064336.0
7662,4689212.0,Sagittarius dSph,283.8313,-30.5453,5.5690,-14.1665,206.5425,43.7457,-3.0,3.84,-0.2,8.23,140.0,147.0,0.03,7.0,5064336.0
7661,17223.0,LMC,80.8941,-69.7561,280.4652,-32.8883,215.7950,-34.1220,9.1,0.41,-1.8,9.42,278.0,17.0,0.05,4.0,5064336.0
7663,3085.0,SMC,13.1583,-72.8002,302.8084,-44.3277,224.2599,-14.8140,8.9,2.14,0.2,8.83,158.0,-37.0,0.06,6.0,5064336.0
7673,4713553.0,BootesI,210.0208,14.5041,358.0829,69.6298,106.2234,19.1997,NaN,NaN,9.5,5.16,99.0,79.0,0.07,4.0,5064336.0


In [115]:
tully_gal_3500[tully_gal_3500['Name'] == 'NGC1316']

,PGC,Name,RAdeg,DEdeg,GLON,GLAT,SGL,SGB,T,Bmag,Ksmag,logK,Vh,VLS,D,e_D,PGC1
477,12651.0,NGC1316,50.6739,-37.2082,240.1629,-56.69,261.045,-38.6292,-1.8,9.29,5.68,11.66,1760.0,1623.0,17.46,7.0,12651.0


In [116]:
tully_gal_3500[tully_gal_3500['Name'] == 'NGC1399']

,PGC,Name,RAdeg,DEdeg,GLON,GLAT,SGL,SGB,T,Bmag,Ksmag,logK,Vh,VLS,D,e_D,PGC1
0,13418.0,NGC1399,54.6212,-35.4506,236.7163,-53.6356,262.546,-42.0776,-4.6,10.35,6.44,11.25,1425.0,1283.0,22.08,11.0,13418.0


In [42]:
z_cf = [z_at_value(Planck15.angular_diameter_distance, d*u.Mpc, zmax = 0.03) if not math.isnan(d) else np.nan for d in tqdm_notebook(tully_3500['D'].values) ]

In [43]:
celery = 299792. # Speed of light

In [44]:
twomrs_g_clusters_masses = []
twomrs_g_clusters_redshifts = []
twomrs_g_clusters_Glon = []
twomrs_g_clusters_Glat = []
twomrs_g_clusters_name = []
twomrs_g_clusters_rvir = []
twomrs_g_clusters_nmem = []
twomrs_g_clusters_twomasxj = []
twomrs_g_clusters_bibz = []
cf_GName = []

for iPGC1, PGC1 in enumerate(tqdm_notebook(tully_3500.PGC1.values)):
    twomrs_g_clusters_masses.append(10**tully_3500.logMK.values[iPGC1])
    twomrs_g_clusters_nmem.append(tully_3500.Mem.values[iPGC1])
    if not np.isnan(z_cf[iPGC1]):
        z_temp = z_cf[iPGC1]
    else:
        z_temp = tully_3500.VLS.values[iPGC1]/celery
    twomrs_g_clusters_redshifts.append(z_temp)
    twomrs_g_clusters_Glon.append(tully_3500.GLON.values[iPGC1])
    twomrs_g_clusters_Glat.append(tully_3500.GLAT.values[iPGC1])
    twomrs_g_clusters_rvir.append(tully_3500.R2t.values[iPGC1])
    
    twomrs_g_clusters_twomasxj.append(np.nan)
    twomrs_g_clusters_bibz.append(np.nan)
    
    twomrs_g_clusters_name.append(tully_gal_3500[tully_gal_3500.PGC == PGC1]['Name'].values[0])

    PGC1_loc_cf = cf_df['PGC1'] == PGC1
    GName = cf_df[PGC1_loc_cf]['GName'].values

    if len(GName) != 0:
        cf_GName.append(cf_df[PGC1_loc_cf]['GName'].values[0])
    else:
        cf_GName.append(np.nan)

## Tully catalog

Do some data munging to match Tully 2MRS and Tully clusters.

In [45]:
# Tully version of 2MRS table
twomrs_df =  pd.read_csv("../DataFiles/Misc/2MRS1175.csv",delimiter="|")
twomrs_df.head()

,pgc,ID_2MASXJ,RAJ,DEJ,Glon,Glat,SGL,SGB,K_c,H_c,...,r_ext,b/a,Flgs,Type,So,Vhel,e_V,C,Bib_z,Name
0,1,23595879+0042066,359.99500,0.70179,96.87975,-59.54168,293.35040,13.42997,11.503,11.882,...,1.473,0.72,0000,98,NN,24478,32,F,20112MRS.FLWO.0000H,23595879+0042066
1,2,00000168+4716282,0.00695,47.27456,113.95532,-14.69903,341.64417,20.73886,9.683,9.969,...,2.001,0.60,0333,3B_T,ZC,5017,9,N,1991RC3.9.C...0000d,UGC_12889
2,5,00000914+3244182,0.03814,32.73844,110.62066,-28.90421,326.17715,19.78053,10.608,10.912,...,1.461,0.80,0000,-4,ZC,10372,19,N,1999PASP..111..438F,IC_5370
3,12,00000865-0622263,0.03601,-6.37399,90.19206,-65.93002,286.42487,11.35101,11.224,11.497,...,1.609,0.26,0000,4,ZC,6531,45,6,20096dF...C...0000J,g0000086-062226
4,14,00000701+0816448,0.02926,8.27913,101.78542,-52.47268,300.91431,15.36726,10.758,11.066,...,1.602,0.60,0666,-5,ZC,11602,0,N,2000ApJS..129..547B,UGC_12890


In [46]:
# North clusters
twomrs_ng_clusters_df = pd.read_csv("../DataFiles/Misc//2MRS1175NG.csv",delimiter="|")

In [47]:
# South clusters
twomrs_sg_clusters_df = pd.read_csv("../DataFiles/Misc//2MRS1175SG.csv",delimiter="|")

In [48]:
# Combine north and south cluster catalogs
twomrs_g_clusters_df = pd.concat([twomrs_ng_clusters_df, twomrs_sg_clusters_df])

In [49]:
# Get unique galaxy groups
twomrs_g_clusters_Nests = pd.Series.unique(twomrs_g_clusters_df['Nest'])
twomrs_g_clusters_PGC1 = []

# Loop over unique galaxy groups and get the brightest central
for Nest in tqdm(twomrs_g_clusters_Nests):
    twomrs_g_clusters_PGC1.append(twomrs_g_clusters_df[twomrs_g_clusters_df['Nest'] == Nest]['PGC1'].values[0])

100%|██████████| 25486/25486 [00:17<00:00, 1479.79it/s]


In [50]:
# Conversion of Vmod into CMB-frame redshift, using eq. 14 of https://arxiv.org/pdf/1307.7213.pdf
# NO LONGER USED

omega_m = 0.27
omega_lambda = 0.73
q0 = 0.5*(omega_m - 2*omega_lambda)
c = 299792.
def Vmod(z, v):
    return c*z*(1+0.5*(1-q0)*z-(1/6.)*(1-q0-3*q0**2+1)*z**2) - v 

In [51]:
# Get relevant cluster property arrays and also cross-match with Tully 2MRS to get names of brightets clusters (this is what is most slow)

for PGC1 in tqdm(twomrs_g_clusters_PGC1):
    
    if PGC1 in tully_3500.PGC1.values: continue # If already in 3500 catalog, ignore

    PGC1_loc = twomrs_g_clusters_df['pgc'] == PGC1
    PGC1_loc_cf = cf_df['PGC1'] == PGC1
    if np.sum(PGC1_loc) == 0: continue
    vgp =  twomrs_g_clusters_df[PGC1_loc]['Vgp'].values[0]
    if vgp <= 3000: continue # Don't trust Tully 2MRS distances in this regime!
        
    twomrs_g_clusters_nmem.append(np.sum(twomrs_g_clusters_df['PGC1'] == PGC1))

    twomrs_g_clusters_masses.append(twomrs_g_clusters_df[PGC1_loc]['L_Mass12'].values[0]*10**12)    
    twomrs_g_clusters_redshifts.append(vgp/celery)
    twomrs_g_clusters_Glon.append(twomrs_g_clusters_df[PGC1_loc]['GLong'].values[0])
    twomrs_g_clusters_Glat.append(twomrs_g_clusters_df[PGC1_loc]['GLat'].values[0])
    twomrs_g_clusters_rvir.append(twomrs_g_clusters_df[PGC1_loc]['R2t'].values[0])
    twomrs_g_clusters_name.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Name'].values[0])
    twomrs_g_clusters_twomasxj.append(twomrs_df[twomrs_df['pgc'] == PGC1]['ID_2MASXJ'].values[0])
    twomrs_g_clusters_bibz.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Bib_z'].values[0])
    GName = cf_df[PGC1_loc_cf]['GName'].values

    if len(GName) != 0:
        cf_GName.append(cf_df[PGC1_loc_cf]['GName'].values[0])
    else:
        cf_GName.append(np.nan)

100%|██████████| 25486/25486 [02:40<00:00, 158.34it/s]


Now loop over halos to get J-factors, cvirs etc from `halo.py`.

In [79]:
# hm_bartels = hl.HaloModel(boost_model='bartels', M_min_halo=1e4*M_s, concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

hm_bartels = hl.HaloModel(boost_model='bartels', alpha='alpha20', concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

# hm_bartels = hl.HaloModel(boost_model='sanchez-conde_tidal', alpha='alpha19', concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

Loading bartels
Interpolating concentration for DarkSky
Using model correa_Planck15


In [80]:
mulog10Jfnb_arr = []
sigmalog10Jfnb_arr = []
mulog10Jf_arr = []
sigmalog10Jf_arr = []

mulog10JBfnb_arr = []
sigmalog10JBfnb_arr = []
mulog10JBf_arr = []
sigmalog10JBf_arr = []

mulog10Df_arr = []
sigmalog10Df_arr = []

twomrs_g_clusters_rvir_der = []
twomrs_g_clusters_cvir_der = []

test = []

# Interpolated cvir errors
cvir_err_arys = np.load("/tigress/nrodd/DM-Catalog-Scan/DataFiles/Misc/cvir_err.npy")
cvir_err_inter = InterpolatedUnivariateSpline(cvir_err_arys[0],cvir_err_arys[1])

Ji = Jpdf.Jc(hm_bartels)
JBi = JBpdf.Jc(hm_bartels)
Di = Dpdf.Dc(hm_bartels)


for i in tqdm_notebook(range(len(twomrs_g_clusters_masses))):
    hm_bartels.cvir = -999
    hm_bartels.rvir = -999
        
    mvir = twomrs_g_clusters_masses[i]*M_s # This is in keV...

    # Some clusters have a 0 mass, if so set mvir to be tiny
    if twomrs_g_clusters_masses[i] == 0.:
        mvir = 0.01*M_s
    
    z = twomrs_g_clusters_redshifts[i]
        
    if 0 < z < 0.03 and np.log10(mvir/M_s) > 10.1: 
        
        cvir = hm_bartels.c_vir(mvir, z)
        rvir = hm_bartels.r_vir(mvir, z)

        mvir /= M_s # Msun

        mulog10M = np.log10(mvir)
        sigmalog10M = mulog10M*0.01

        mulog10c = np.log10(cvir)
        #sigmalog10c = 0.15 # NB: Might want to update this later!    
        sigmalog10c = cvir_err_inter(np.log10(mvir))

        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J = Ji.log10Jmusig(mulog10M, sigmalog10M, mulog10c, sigmalog10c, z)
        mulog10JBnb, sigmalog10JBnb, mulog10JB, sigmalog10JB = JBi.log10Jmusig(mulog10M, sigmalog10M, mulog10c, sigmalog10c, z)
        mulog10D, sigmalog10D = Di.log10Dmusig(mulog10M, sigmalog10M, z)
    
    else:
        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J = -999., -999., -999., -999.
        mulog10JBnb, sigmalog10JBnb, mulog10JB, sigmalog10JB = -999., -999., -999., -999.
        mulog10D, sigmalog10D = -999., -999.
        cvir, rvir = -999, -999
        
    mulog10Jfnb_arr.append(mulog10Jnb)
    sigmalog10Jfnb_arr.append(sigmalog10Jnb)
    mulog10Jf_arr.append(mulog10J)
    sigmalog10Jf_arr.append(sigmalog10J)
    
    mulog10JBfnb_arr.append(mulog10JBnb)
    sigmalog10JBfnb_arr.append(sigmalog10JBnb)
    mulog10JBf_arr.append(mulog10JB)
    sigmalog10JBf_arr.append(sigmalog10JB)
    
    mulog10Df_arr.append(mulog10D)
    sigmalog10Df_arr.append(sigmalog10D)
    
    twomrs_g_clusters_rvir_der.append(rvir)
    twomrs_g_clusters_cvir_der.append(cvir)

In [81]:
print np.min(mulog10Jf_arr), np.max(mulog10Jf_arr)

-999.0 23.990768547


Make dataframe and add in information we just inferred

In [82]:
nhalos = len(mulog10Jf_arr)

sales = zip(twomrs_g_clusters_name,
            mulog10Jf_arr,
            sigmalog10Jf_arr,
            mulog10Jfnb_arr,
            sigmalog10Jfnb_arr,
            mulog10JBf_arr,
            sigmalog10JBf_arr,
            mulog10JBfnb_arr,
            sigmalog10JBfnb_arr,
            mulog10Df_arr,
            sigmalog10Df_arr,
            np.log10(twomrs_g_clusters_masses),
            twomrs_g_clusters_redshifts,
            twomrs_g_clusters_Nests,
            twomrs_g_clusters_nmem,
            twomrs_g_clusters_Glon,
            twomrs_g_clusters_Glat,
            twomrs_g_clusters_cvir_der,
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc),
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc)/np.array(twomrs_g_clusters_cvir_der),
            twomrs_g_clusters_twomasxj,
            twomrs_g_clusters_bibz,
            cf_GName)
labels = ["Name", "mulog10J_inf","siglog10J_inf","mulog10Jnb_inf","siglog10Jnb_inf",
          "mulog10JB_inf","siglog10JB_inf","mulog10JBnb_inf","siglog10JBnb_inf",
          "mulog10D_inf","siglog10D_inf",
          "log10Mvir_inf", "z", "Nest", "Ng", "l", "b", "cvir_inf", "rvir_inf", "rs", "ID_2MASXJ", "Bib_z", "GName"]
df = pd.DataFrame.from_records(sales, columns=labels)
df = df[df["z"] < 0.03] # Cut to avoid the issue with crazy massive halos from overincluding satellite contribution
df = df[df["mulog10J_inf"] != -999.] 
df = df[df["Name"] != "Milky Way"] # Remove Milky Way
dftop = df.sort_values(["mulog10J_inf"],ascending=False)[:nhalos]

Add 3FGL-related information

In [83]:
# Load in the 3FGL catalog 
source_3fg_df = pd.read_csv('../DataFiles/Catalogs//3fgl.dat', sep='|', comment='#')

In [84]:
# Remove whitespace from column names
source_3fg_df.rename(columns=lambda x: x.strip(), inplace=True)
for col in source_3fg_df.columns.values:
    try:
        source_3fg_df[col] = source_3fg_df[
            col].map(str.strip)
    except TypeError:
        continue

# Convert to numeric data
source_3fg_df = source_3fg_df.convert_objects(
    convert_numeric=True)

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':


In [85]:
# Change Virgo and Fornax centers

dftop.loc[dftop['GName'] == 'Virgo','l'] = 283.7777978
dftop.loc[dftop['GName'] == 'Virgo','b'] = 74.4911308

dftop.loc[dftop['GName'] == 'Fornax','l'] = 236.72
dftop.loc[dftop['GName'] == 'Fornax','b'] = -53.64

In [86]:
# Astropy-formatted coordinates of cluster and 3FGL
c2 = SkyCoord("galactic", l=dftop['l']*u.deg, b=dftop['b']*u.deg)
c3 = SkyCoord("galactic", l=source_3fg_df['_Lii']*u.deg, b=source_3fg_df['_Bii']*u.deg)

In [87]:
# Select 3FGL sources within some radius

threefgl0p5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, .5*u.deg)
for i in range(len(dftop)):
    threefgl0p5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl1deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 1*u.deg)
for i in range(len(dftop)):
    threefgl1deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl2deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 2*u.deg)
for i in range(len(dftop)):
    threefgl2deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)
    
threefgl5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 5*u.deg)
for i in range(len(dftop)):
    threefgl5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)
    
threefgl10deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 10*u.deg)
for i in range(len(dftop)):
    threefgl10deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

In [88]:
# 3FGL sources within x degrees
dftop['3FGL 0.5'] = pd.Series(threefgl0p5deg, index=dftop.index)
dftop['3FGL 1'] = pd.Series(threefgl1deg, index=dftop.index)
dftop['3FGL 2'] = pd.Series(threefgl2deg, index=dftop.index)
dftop['3FGL 5'] = pd.Series(threefgl5deg, index=dftop.index)
dftop['3FGL 10'] = pd.Series(threefgl10deg, index=dftop.index)

In [89]:
# Number of 3FGL sources within 5 degrees
dftop['N3FGL 5'] = pd.Series([len(x) for x in threefgl5deg], index=dftop.index)

In [90]:
# Angular extension

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=hm_bartels.h*100, Om0=hm_bartels.omega_m)
ang_ext = 2*dftop['rs'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)
rvir_ang = dftop['rvir_inf'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)

dftop['ang_ext'] = pd.Series(ang_ext, index=dftop.index)
dftop['theta_vir'] = pd.Series(rvir_ang, index=dftop.index)
# dftop['rvir_ang'] = pd.Series(rvir_ang, index=dftop.index)

In [91]:
# M200
dftop['log10M200_inf'] = pd.Series(np.log10(np.array([hm_bartels.M200_cvir(m, z, c) for m,z,c in zip(10**dftop['log10Mvir_inf'].values*M_s, dftop['z'], dftop['cvir_inf'])])/M_s), index=dftop.index)

In [92]:
dftop['ra'] = c2.transform_to('icrs').ra.degree  
dftop['dec'] = c2.transform_to('icrs').dec.degree  

In [93]:
# dftop['bsh'] = 10**dftop.mulog10J_inf/10**dftop.mulog10Jnb_inf

In [94]:
# Boost factor (1+bsh)
# dftop['bsh'] = pd.Series([1+hm_bartels.bsh(m, z).flatten()[0] for m,z in zip(10**dftop['log10Mvir_inf']*M_s,dftop['z'])], index=dftop.index)

In [95]:
dftop.columns.values

array(['Name', 'mulog10J_inf', 'siglog10J_inf', 'mulog10Jnb_inf',
       'siglog10Jnb_inf', 'mulog10JB_inf', 'siglog10JB_inf',
       'mulog10JBnb_inf', 'siglog10JBnb_inf', 'mulog10D_inf',
       'siglog10D_inf', 'log10Mvir_inf', 'z', 'Nest', 'Ng', 'l', 'b',
       'cvir_inf', 'rvir_inf', 'rs', 'ID_2MASXJ', 'Bib_z', 'GName',
       '3FGL 0.5', '3FGL 1', '3FGL 2', '3FGL 5', '3FGL 10', 'N3FGL 5',
       'ang_ext', 'theta_vir', 'log10M200_inf', 'ra', 'dec'], dtype=object)

In [96]:
dftop[dftop['GName'] == 'Virgo']

,Name,mulog10J_inf,siglog10J_inf,mulog10Jnb_inf,siglog10Jnb_inf,mulog10JB_inf,siglog10JB_inf,mulog10JBnb_inf,siglog10JBnb_inf,mulog10D_inf,...,3FGL 1,3FGL 2,3FGL 5,3FGL 10,N3FGL 5,ang_ext,theta_vir,log10M200_inf,ra,dec
210,NGC4472,19.796148,0.352051,18.372857,0.338266,19.545291,0.381305,18.121999,0.368616,20.329794,...,[3FGL J1230.9+1224],"[3FGL J1223.2+1215, 3FGL J1230.9+1224, 3FGL J1...","[3FGL J1223.3+0818, 3FGL J1223.2+1215, 3FGL J1...","[3FGL J1222.4+0414, 3FGL J1239.5+0443, 3FGL J1...",7,2.311796,7.355798,14.567979,187.705935,12.391097


In [97]:
dftop['dA'] = Planck15.angular_diameter_distance(dftop['z'])

In [108]:
np.load("../Scan-Small-ROI/data/Tully/LL2_TSmx_lim_b_nb_o6_data.npz")['lim']

array([  2.48794565e-25,   2.54236460e-25,   2.75054817e-25,
         3.06732165e-25,   3.46320257e-25,   4.43623225e-25,
         5.62954044e-25,   7.02589513e-25,   8.59681937e-25,
         1.03263794e-24,   1.21777861e-24,   1.41299631e-24,
         1.61440478e-24,   1.82221898e-24,   2.03544568e-24,
         2.25365267e-24,   2.47542157e-24,   2.70071470e-24,
         3.16648016e-24,   3.63619887e-24,   4.11499544e-24,
         4.59858873e-24,   5.09242618e-24,   5.58636439e-24,
         6.08078441e-24,   6.83214782e-24,   7.58789651e-24,
         8.61088374e-24,   9.89691754e-24,   1.11972225e-23,
         1.25170646e-23,   1.38601324e-23,   1.52303504e-23,
         1.66260572e-23,   1.80471423e-23,   1.94623987e-23,
         2.23175295e-23,   2.51551766e-23,   2.80184621e-23,
         3.09066490e-23,   3.38622350e-23,   3.99871935e-23,
         4.63208610e-23,   5.63660558e-23,   7.43692744e-23,
         9.40648652e-23,   1.37625210e-22,   1.86066399e-22,
         2.39351563e-22,

In [109]:
dftop[['GName','Name', 'mulog10J_inf','log10Mvir_inf','z','l','b','cvir_inf','theta_vir']]

,GName,Name,mulog10J_inf,log10Mvir_inf,z,l,b,cvir_inf,theta_vir
3072,NaN,Andromeda,20.496300,12.441000,0.000170,121.505400,-21.788600,11.005717,28.262611
210,Virgo,NGC4472,19.796148,14.656000,0.003577,283.777798,74.491131,6.363708,7.355798
2565,NaN,NGC5128,19.591623,12.950000,0.000821,307.884500,17.081400,9.832124,8.628832
3211,NaN,NGC0253,19.460277,12.775000,0.000789,98.237100,-87.890300,10.227946,7.847010
3239,NaN,Maffei 1,19.387875,12.690000,0.000783,136.231500,-0.440500,10.397845,7.415162
3074,NaN,NGC 6822,19.303651,10.709000,0.000108,25.341100,-18.395100,15.127216,11.686972
2809,NaN,NGC3031,19.288635,12.630000,0.000826,141.879100,40.866200,10.553715,6.712719
0,Centaurus,NGC4696,19.026917,14.624000,0.008438,302.224100,21.646500,6.398877,3.056262
906,NaN,NGC1399,18.996182,13.869000,0.004108,236.623100,-53.883800,7.834312,3.502262
2153,NaN,IC0356,18.962055,13.574000,0.003144,138.061500,12.695500,8.447134,3.645938


In [99]:
dftop.to_csv("../DataFiles/Catalogs/2MRSLocalTully_ALL_DATAPAPER_Planck15_alpha20_v7.csv")

In [489]:
# dftop[['Name','GName','mulog10J_inf','mulog10Jnb_inf','log10Mvir_inf','cvir_inf','rvir_inf','l','b','3FGL 0.5','dA']]

In [285]:
# dftop.to_csv("../DataFiles/Catalogs/2MRSTully_ALL_DATAPAPER_Planck15_sanchezboost_v5.csv")

In [490]:
# dfless8.to_csv("2MRSTully_8kpc.csv")